<a href="https://colab.research.google.com/github/eak0mo/GNB-eqipment-monitoring/blob/main/sql_ml_maintanance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading kaggle datatset and expanding from zip

In [6]:
!pip install -q kaggle
!pip install -q lazypredict

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
kaggle_path ="/content/drive/MyDrive/kaggle/"

In [8]:
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
!kaggle datasets download -d dnkumars/industrial-equipment-monitoring-dataset

Dataset URL: https://www.kaggle.com/datasets/dnkumars/industrial-equipment-monitoring-dataset
License(s): apache-2.0
  0% 0.00/291k [00:00<?, ?B/s]
100% 291k/291k [00:00<00:00, 49.8MB/s]


In [11]:
! mkdir kaggle_data
! unzip {"industrial-equipment-monitoring-dataset.zip"} -d kaggle_data

Archive:  industrial-equipment-monitoring-dataset.zip
  inflating: kaggle_data/equipment_anomaly_data.csv  


In [12]:
drive.flush_and_unmount()

data preprocessing + sql

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split as split
from sklearn.naive_bayes import GaussianNB
import pickle
import sqlite3
from lazypredict.Supervised import LazyClassifier, LazyRegressor
from sklearn.preprocessing import LabelEncoder

In [13]:
data=pd.read_csv("/content/kaggle_data/equipment_anomaly_data.csv")

In [ ]:
conn = sqlite3.connect('test.sqlite')
cursor=conn.cursor()

In [ ]:
data.to_sql('test', conn, if_exists='replace', index=False)

7672

In [ ]:
cursor.execute("SELECT * FROM test WHERE faulty=0")
tables = cursor.fetchall()
print(len(tables))

6905


In [15]:
# split original data into 2, train_test + use cast sim (60%,30%,10%)
# use train_test split for the first one
X=data.drop(['faulty'],axis=1)
y=data['faulty']


In [28]:
x_train, x_test, y_train, y_test = split(X, y, test_size=0.35, random_state=40)
x_train_g, x_test_g, y_train_g, y_test_g = split(X, y, test_size=0.25, random_state=100)

In [58]:
#encoding
encoder = LabelEncoder()
x_train_g['equipment'] = encoder.fit_transform(x_train_g['equipment'])
x_test_g['equipment'] = encoder.transform(x_test_g['equipment'])
x_train_g['location'] = encoder.fit_transform(x_train_g['location'])
x_test_g['location'] = encoder.transform(x_test_g['location'])


Ml model training and testing

In [23]:
classifier = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = classifier.fit(x_train, x_test, y_train, y_test)
models

 59%|█████▉    | 19/32 [00:05<00:02,  4.46it/s]/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
 97%|█████████▋| 31/32 [00:07<00:00,  7.07it/s]/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
100%|██████████| 32/32 [

[LightGBM] [Info] Number of positive: 493, number of negative: 4493
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1036
[LightGBM] [Info] Number of data points in the train set: 4986, number of used features: 12
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.098877 -> initscore=-2.209767
[LightGBM] [Info] Start training from score -2.209767


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
GaussianNB,0.99,0.94,0.94,0.99,0.05
LGBMClassifier,0.98,0.93,0.93,0.98,0.25
SVC,0.98,0.93,0.93,0.98,0.24
XGBClassifier,0.98,0.93,0.93,0.98,0.27
RandomForestClassifier,0.98,0.92,0.92,0.98,1.12
BaggingClassifier,0.98,0.91,0.91,0.98,0.47
DecisionTreeClassifier,0.97,0.91,0.91,0.97,0.12
ExtraTreesClassifier,0.98,0.89,0.89,0.98,0.43
AdaBoostClassifier,0.97,0.88,0.88,0.97,0.45


In [26]:
predictions

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
GaussianNB,0.99,0.94,0.94,0.99,0.05
LGBMClassifier,0.98,0.93,0.93,0.98,0.25
SVC,0.98,0.93,0.93,0.98,0.24
XGBClassifier,0.98,0.93,0.93,0.98,0.27
RandomForestClassifier,0.98,0.92,0.92,0.98,1.12
BaggingClassifier,0.98,0.91,0.91,0.98,0.47
DecisionTreeClassifier,0.97,0.91,0.91,0.97,0.12
ExtraTreesClassifier,0.98,0.89,0.89,0.98,0.43
AdaBoostClassifier,0.97,0.88,0.88,0.97,0.45


In [40]:
#building a guassian nb model specifically
gnb_model = GaussianNB()
gnb_model.fit(x_train_g, y_train_g)
predict_g= gnb_model.predict(x_test_g)

accuracy_g = gnb_model.score(x_test_g, y_test_g)
print(accuracy_g)

0.9833159541188738


Packaging

In [55]:
#!mv gnb_encoder.pkl /content/drive/MyDrive/kaggle
#!mv gnb_model.pkl /content/drive/MyDrive/kaggle
load_path= "/content/drive/MyDrive/kaggle"

In [51]:
#save encoder
with open('gnb_encoder.pkl', 'wb') as f:
    pickle.dump(encoder, f)

In [52]:
# save gaussian model
with open('gnb_model.pkl', 'wb') as f:
    pickle.dump(gnb_model, f)

In [57]:
with open('/content/drive/MyDrive/kaggle/gnb_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [56]:
with open('/content/drive/MyDrive/kaggle/gnb_encoder.pkl', 'rb') as f:
    loaded_encoder = pickle.load(f)